# Setup

In [ ]:
import sys
import platform

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn as sk
import torch

has_gpu = torch.cuda.is_available()
has_mps = torch.backends.mps.is_built()
custom_device = "mps" if has_mps else "cuda" if torch.cuda.is_available() else "cpu"

print(f"Python Platform: {platform.platform()}")
print(f"PyTorch Version: {torch.__version__}\n")
print(f"Python {sys.version}")
print(f"Pandas {pd.__version__}")
print(f"Numpy {np.__version__}")
print(f"Scikit-Learn {sk.__version__}")
print("NVIDIA/CUDA GPU is", "available" if has_gpu else "NOT AVAILABLE")
print("MPS (Apple Metal) is", "AVAILABLE" if has_mps else "NOT AVAILABLE")

print(f"\nCustome Device:\t{custom_device}")

# Option 1 on Mac (with Apple Silicon) is to use the CPU:
#torch.set_default_device("cpu") # <- setting it manually to "cpu"

# Option 2 on Mac (with Apple Silicon) is to use MPS:
torch.set_default_device(custom_device)

print(f"Active device:\t{torch.get_default_device()}")

# Testing

print("\nRun test:")
layer = torch.nn.Linear(20,30)
print(f"\tLayer weights are on device: {layer.weight.device}")
print(f"\tLayer creating data on device: {layer(torch.randn(128,20)).device}")

# Classification